# 5 УРОК

ПОДЗАПРОСЫ, ПРЕДСТАВЛЕНИЯ, СОЗДАНИЕ ТАБЛИЦ

## 1

С помощью какого оператора в ClickHouse можно создать представление?

Возможно несколько вариантов ответа.

### Ответ:
    CREATE VIEW
    
    CREATE MATERIALIZED VIEW

## 2 
В чём заключается различие между материализованным и обычным представлением? 

### Ответ:
    
Материализованное хранит данные на диске

Обычное сохраняет запрос, который выполняется при обращении

## 3

In [ ]:
Данные для следующих задач: Airbnb в Берлине. Не забывайте о преобразовании столбцов к нужному типу данных!

Note: Как зайти в ClickHouse. Если таблицы не видны в списке – попробуйте выйти и зайти ещё раз.

Имеются следующие таблицы:

1. listings – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; 

поскольку столбцов очень много, нужные перечислены в текстах самих задач

2. calendar_summary – информация о доступности и цене того или иного жилья по дням

listing_id – идентификатор жилья (объявления)
date – дата
available – доступность жилья в данный день (t/f)
price – цена за ночь
3. reviews – отзывы

listing_id –  идентификатор жилья  
id – id отзыва
date – дата отзыва
reviewer_id – id ревьюера (автора отзыва)
reviewer_name – имя автора
comments – сам отзыв

## 4

In [ ]:
Подзапросы можно использовать в разных частях запроса, в том числе и в блоке WHERE. Потренируемся!

-- пример
SELECT
    column_1,
    column_2
FROM
    table
WHERE
    column_2 = (SELECT MAX(column_2) 
                FROM table)


Сначала оставьте только те объявления, в которых оценка на основе отзывов выше среднего, 

а число отзывов в месяц составляет строго меньше трёх. Затем отсортируйте по убыванию две колонки: 
    
сначала по числу отзывов в месяц, потом по оценке. В качестве ответа укажите id объявления из первой строки.

review_scores_rating – оценка на основе отзывов
reviews_per_month – число отзывов в месяц
id – идентификатор объявления
Таблица – listings. Наивысшие показатели = сортировка по убыванию.

ВАЖНОЕ ЗАМЕЧАНИЕ!

Если вы делаете задание через REDASH, убедитесь, 
что в ответ вносите значение без запятых между порядками числа ( 9,424,234.05 -> 9424234.05 )

In [ ]:
# НЕКОРЕКТНЫЙ ВАРИАНТ
# SELECT
#     id,
#     review_scores_rating,
#     reviews_per_month
# FROM
#     listings
# WHERE
#     toFloat64OrNull(review_scores_rating) > (SELECT 
# 					AVG(toFloat64OrNull(review_scores_rating))
#                 FROM listings)
# 		AND reviews_per_month < 3 

# ORDER BY  reviews_per_month DESC, review_scores_rating DESC
# LIMIT 100

In [ ]:
# 15947545	97.0	2.99
# 22476940	100.0	2.99
# 24145948	99.0	2.98
# 21390172	99.0	2.98
# 25364559	97.0	2.98
# 914944	97.0	2.98
# 4149532	100.0	2.98
# 15185336	98.0	2.97
# 24785452	100.0	2.97
# 25455082	99.0	2.96

In [ ]:
# ПРАВИЛЬНЫЙ ВАРИАНТ
SELECT
    id,
    toFloat64OrNull(review_scores_rating) as rating,
    reviews_per_month
FROM
    listings
WHERE
     rating > (SELECT 
					AVG(toFloat64OrNull(review_scores_rating))
                FROM listings
               )
		AND reviews_per_month < 3 

ORDER BY  reviews_per_month DESC, rating DESC
LIMIT 100

In [ ]:
22476940	100	2.99
15947545	97	2.99
4149532	100	2.98
24145948	99	2.98
21390172	99	2.98
25364559	97	2.98
914944	97	2.98
24785452	100	2.97
15185336	98	2.97

##### Ответ: 22476940
Странно но ответ был на 2й позиции в неправильном варианте??

## 5

In [ ]:
Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, 

для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату,

которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем. 

id – идентификатор объявления

host_id – идентификатор хозяина

room_type – тип жилья ('Private room')

latitude – широта

longitude – долгота

52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

В качестве ответа укажите идентификатор хозяина (host_id), сдающего данную комнату.

geoDistance(13.4050, 52.5200, долгота, широта)

Note: задание можно решить как используя подзапрос в WHERE, так и с помощью WITH. 

    При подсчете среднего тип комнаты тоже следует учесть.

In [ ]:
SELECT
    id,
    host_id,
    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance
    
FROM
    listings
WHERE
     distance < 
    (SELECT 
			AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)))
            FROM listings
            WHERE room_type  = 'Private room' )
		AND room_type  = 'Private room'

ORDER BY  distance DESC
LIMIT 100

### ВАРИАНТ с WITH

In [ ]:
WITH
    (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)))
            FROM listings WHERE room_type  = 'Private room') AS avg_distance
 

SELECT
    id,
    host_id,
    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance
    
FROM
    listings
WHERE
     distance < avg_distance
		AND room_type  = 'Private room'

ORDER BY  distance DESC
LIMIT 100

In [ ]:
22306678	163065208	4608.982
19843597	54900115	4608.205
7172034	25835088	4608.166
20508256	21346093	4607.219
2398968	12258154	4606.447
7732154	19493279	4605.592
13648651	79241577	4605.3716

###### ОТВЕТ : 163065208

## 6

In [ ]:
Представим, что вы планируете снять жилье в Берлине на 7 дней, используя более хитрые фильтры, чем предлагаются на сайте.

В этой задаче можно потренироваться в написании подзапросов, но задание можно решить и без них.

Отберите объявления из таблицы listings, которые:

находятся на расстоянии от центра меньше среднего (здесь нам пригодится запрос из предыдущего задания)

обойдутся дешевле 100$ в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю, 
                               т.е его нужно делить на кол-во дней)

имеют последние отзывы (last_review), начиная с 1 сентября 2018 года

имеют WiFi в списке удобств (amenities)

Отсортируйте полученные значения по убыванию review_scores_rating (не забудьте перевести строку к численному виду) 

и в качестве ответа укажите host_id из первой строки. 

Hint: документация по функции multiSearchAnyCaseInsensitive(колонка, ['искомая_подстрока']), 
    
    которая ищет заданную подстроку в строке вне зависимости от регистра и выдает 0 в случае, если подстрока не была найдена

In [ ]:
WITH
    (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)))
            FROM listings WHERE room_type  = 'Private room') AS avg_distance
 

SELECT
    id,
    host_id,
    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance,
    ROUND(toFloat64OrNull(replaceRegexpAll(price, '[$,]', '')) + toFloat64OrNull(replaceRegexpAll(cleaning_fee, '[$,]', ''))/7 , 2) as PriceALL_1_day,
    amenities,
    last_review,
    toFloat64OrNull(review_scores_rating) AS rating
    
FROM
    listings
WHERE
        distance < avg_distance
    AND room_type  = 'Private room'
    AND PriceALL_1_day < 100
    AND multiSearchAnyCaseInsensitive(amenities, ['WiFi']) = True
    AND  last_review > '2018-09-01'

ORDER BY  rating DESC
LIMIT 100

In [ ]:
28071741	39740677	2528.3916	53.57	{Wifi,Kitchen,Elevator,Heating,Washer,Dryer,"Smoke detector","Fire extinguisher",Essentials,"Lock on bedroom door",Hangers,Iron,"Private living room","Coffee maker",Refrigerator,Dishwasher,"Dishes and silverware","Cooking basics",Oven,Stove,"Patio or balcony"}
12967814	41119131	3603.6045	82.29	{TV,"Cable TV",Internet,Wifi,"Air conditioning",Kitchen,"Pets live on this property",Dog(s),"Free street parking","Indoor fireplace","Buzzer/wireless intercom",Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Carbon monoxide detector","First aid kit","Fire extinguisher",Essentials,Shampoo,Hangers,"Hair dryer",Iron,"translation missing: en.hosting_amenity_49","Private entrance","Hot water","Patio or balcony","Luggage dropoff allowed","Long term stays allowed","Host greets you"}
5271228	27285243	4371.121	50	{Internet,Wifi,Kitchen,"Pets allowed",Heating,"Family/kid friendly",Washer,Essentials,Shampoo,"Lock on bedroom door","Hair dryer","Laptop friendly workspace","Hot water",Other}
329648	1682922	3645.9812	50.14	{TV,Internet,Wifi,Kitchen,"Paid parking off premises","Buzzer/wireless intercom",Heating,Washer,Essentials,Shampoo,Hangers,"Hair dryer",Iron,"Laptop friendly workspace",Bathtub,"Hot water","Bed linens","Extra pillows and blankets",Refrigerator,Dishwasher,"Dishes and silverware","Cooking basics",Oven,Stove,"Single level home","Patio or balcony","Host greets you"}
28069928	46659004	3760.4146	23	{Wifi,Kitchen,"Smoking allowed","Free street parking",Heating,Washer,"First aid kit",Essentials,Hangers,"Laptop friendly workspace","Hot water","Bed linens","Extra pillows and blankets",Refrigerator,Dishwasher,"Dishes and silverware","Cooking basics",Oven,Stove}
11508748	60635775	2641.1787	63.29	{TV,Wifi,"Paid parking off premises",Elevator,Heating,Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer","Laptop friendly workspace","Hot water","Bed linens","Extra pillows and blankets",Microwave,"Coffee maker",Refrigerator,Dishwasher,"Dishes and silverware","Patio or balcony","Garden or backyard","Luggage dropoff allowed","Wide hallway clearance","Step-free access","Wide doorway","Flat path to front door","Well-lit path to entrance","Step-free access","Accessible-height bed","Step-free access","Accessible-height toilet","Host greets you","Handheld shower head"}
4246694	7012464	4469.4087	34.14	{Internet,Wifi,Kitchen,"Smoking allowed","Free street parking","Buzzer/wireless intercom",Heating,Washer,Essentials,Shampoo,Hangers,"Hair dryer",Iron,"Hot water","Bed linens","Extra pillows and blankets","Coffee maker",Refrigerator,Dishwasher,"Dishes and silverware","Cooking basics",Oven,Stove,"Garden or backyard","Host greets you"}

###### ОТВЕТ : 39740677

## 7

In [ ]:
Давайте найдем в таблице calendar_summary те доступные (available='t') объявления,

у которых число отзывов от уникальных пользователей в таблице reviews выше среднего.

NB! Для простоты будем считать, что отзыв — это уникальный посетитель на уникальное жилье,

не учитывая возможные повторные отзывы от того же посетителя.

Для этого с помощью конструкции WITH посчитайте среднее число уникальных reviewer_id из таблицы reviews на каждое жильё,

потом проведите джойн таблиц calendar_summary и reviews по полю listing_id 

(при этом из таблицы calendar_summary должны быть отобраны уникальные listing_id, отфильтрованные по правилу available='t').

Результат отфильтруйте так, чтобы остались только записи, у которых число отзывов от уникальных людей выше среднего.

Отсортируйте результат по возрастанию listing_id и в качестве ответа впишите количество отзывов от уникальных пользователей 

из первой строки.

Hint: для решения проблем со вложенными агрегационными функциями (и агрегационными функциями там, где их не ждут) 

как раз очень помогут подзапросы.

In [ ]:
WITH(SELECT AVG(Count_reviewer)
    FROM (SELECT listing_id,COUNT(DISTINCT reviewer_id) AS Count_reviewer
        FROM reviews
        GROUP BY listing_id
        ) 
    ) AS Avg_Count_reviewer

SELECT DISTINCT(l.listing_id) AS listing_id,
        COUNT(DISTINCT(r.reviewer_id)) AS Count_reviewer,
        Avg_Count_reviewer
FROM calendar_summary  AS l 
    LEFT JOIN reviews AS r
    ON (l.listing_id = r.listing_id)
WHERE available = 't'
GROUP BY listing_id
HAVING Count_reviewer > Avg_Count_reviewer
ORDER BY listing_id
LIMIT 100

In [ ]:
# # ИЗ РАЗБОРА ПОМОЩЬ
# WITH
# (SELECT 
# 	AVG(column1)
#  FROM 
# 	(SELECT 
# 			COUNT(DISTINCT(column1)) AS column1,
# 			column2
#    FROM 
# 			table_name
# 	 GROUP BY 
# 			column2)) AS alias

In [ ]:
# ИЗ РАЗБОРА ПОМОЩЬ
# WITH
# (SELECT AVG(review_num) FROM (SELECT COUNT(DISTINCT(reviewer_id)) review_num, listing_id  FROM reviews GROUP BY listing_id)) AS avg_review
# SELECT
# listing_id,
# review_num
# FROM (
# 	SELECT
#     	COUNT(DISTINCT(reviewer_id)) review_num,
#     	listing_id
# 	FROM reviews
# 	GROUP BY listing_id
# 	) AS r
# JOIN (
# 	SELECT
#     	listing_id
# 	FROM calendar_summary
# 	WHERE available='t' 
# GROUP BY listing_id
# 	) AS c
# ON
# 	c.listing_id = r.listing_id
# WHERE review_num > avg_review  
# ORDER BY listing_id
# LIMIT 10

In [ ]:
2015	118	21.399077154201095
3176	142	21.399077154201095
3309	25	21.399077154201095
7071	195	21.399077154201095
14325	23	21.399077154201095
16644	48	21.399077154201095
17409	275	21.399077154201095
17904	218	21.399077154201095

##### ОТВЕТ: 118

## 8

In [ ]:
Возвращаемся к данными мобильного приложения!

installs — содержит данные об установках приложения по дням.

DeviceID — идентификатор устройства, на которое было установлено приложение;
InstallationDate — дата установки приложения;
InstallCost — цена установки приложения в рублях;
Platform — платформа, на которой было установлено приложение (iOS/ Android);
Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).
events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

DeviceID — идентификатор устройства, на котором используется приложение;
AppPlatform — платформа, на которой используется приложение (iOS/ Android);
EventDate — дата, за которую собрана статистика;
events — количество просмотров всех товаров за этот день у этого DeviceID.
checks — содержит данные о покупках пользователей в приложении по дням

UserID — идентификатор пользователя;
Rub — суммарный чек пользователя на дату;
BuyDate — дата, за которую собрана статистика.
 devices – чтобы просмотры и установки можно было объединить с покупками

DeviceID — идентификатор устройства;
UserID — идентификатор пользователя.

## 9

In [ ]:
С помощью оператора CASE в SQL можно составлять условные конструкции, выполнять проверку условий и возвращать результат в зависимости от их выполнения. Самих условий под оператором WHEN может быть сколько угодно, конструкцию необходимо завершить оператором END. Структура выглядит следующим образом:

 

Если у Вас есть вопросы, напишите сюда.

SELECT 
    CASE
        -- если значение в column_1 больше, чем в column_2, в новый столбец будет записана единичка
        WHEN column_1 > column_2 THEN 1 
        -- если значение в column_1 меньше, чем в column_2, в новый столбец будет записана двойка
        WHEN column_1 < column_2 THEN 2
        -- во всех остальных случаях – тройка
        ELSE 3
    END AS cond_1 -- END – завершаем конструкцию, а новый столбец называем cond_1
FROM
    table


Какие из перечисленных условных конструкций составлены некорректно?

###### ОТВЕТ:
    WHERE price > 1000 THEN 'ok' ELSE 'not ok' END  - НЕ КОРРЕКТНО !!
    
    CASE WHEN price>1000 THEN 'ok' WHEN price<500 THEN 'not ok' ELSE 'normal' END - ЭТО ПРАВИЛЬНО !!
    
    CASE WHEN price > 1000 THEN 'ok' ELSE 'not ok'  - НЕ КОРРЕКТНО !!
    

## 10

In [ ]:
Теперь задача! Используйте таблицу checks и разделите всех покупателей на сегменты:

NB! Правые границы берутся не включительно, например, чек в 10 рублей будет относиться к сегменту С

А — средний чек покупателя менее 5 ₽
B — средний чек покупателя от 5-10 ₽
C — средний чек покупателя от 10-20 ₽
D — средний чек покупателя от 20 ₽
Отсортируйте результирующую таблицу по возрастанию UserID и укажите сегмент четвертого пользователя.

In [ ]:
SELECT UserID,
    CASE
        WHEN mean_rub < 5 THEN 'A'
        WHEN mean_rub < 10 THEN 'B'
        WHEN mean_rub < 20 THEN 'C'
        ELSE 'D'
    END AS segment 
FROM
    (SELECT UserID,
                AVG(Rub) AS mean_rub
    FROM checks
    GROUP BY  UserID
    ) 
    
ORDER BY UserID
LIMIT 10

In [ ]:
1406814346437	A
3116519148596	B
3827093218449	B
4301734536835	C
9820728992879	A
10995267265098	A
13652596829677	A
20964545223437	A

In [ ]:
# Этот вариант кажется проще )) тоже мой придумал опосля
SELECT UserID,
    AVG(Rub),
    CASE
        WHEN AVG(Rub) < 5 THEN 'A'
        WHEN AVG(Rub) < 10 THEN 'B'
        WHEN AVG(Rub) < 20 THEN 'C'
        ELSE 'D'
    END AS segment 
FROM checks
GROUP BY UserID
ORDER BY UserID
LIMIT 10

In [ ]:
1406814346437	3.5	A
3116519148596	5	B
3827093218449	5.5	B
4301734536835	10	C
9820728992879	1	A
10995267265098	0	A

##### ОТВЕТ: C

## 11

In [ ]:
Используйте предыдущий запрос как подзапрос и посчитайте, 

сколько клиентов приходится на каждый сегмент и сколько доходов он приносит. 

Отсортируйте результат по убыванию суммы доходов на сегмент и в качестве ответа укажите наибольшую сумму.

In [ ]:
SELECT r.segment,
        SUM(l.Rub) AS SUM_segment
FROM checks AS l
JOIN
    (SELECT UserID,
        AVG(Rub),
        CASE
            WHEN AVG(Rub) < 5 THEN 'A'
            WHEN AVG(Rub) < 10 THEN 'B'
            WHEN AVG(Rub) < 20 THEN 'C'
            ELSE 'D'
        END AS segment 
    FROM checks
    GROUP BY UserID
    ) AS r
ON l.UserID = r.UserID
GROUP BY r.segment
ORDER BY SUM_segment DESC

In [ ]:
# Добавил COUNT(l.UserID) AS Count_UserID,
# SELECT r.segment,
#         COUNT(l.UserID) AS Count_UserID,
#         SUM(l.Rub) AS SUM_segment
# FROM checks AS l
# JOIN
#     (SELECT UserID,
#         AVG(Rub),
#         CASE
#             WHEN AVG(Rub) < 5 THEN 'A'
#             WHEN AVG(Rub) < 10 THEN 'B'
#             WHEN AVG(Rub) < 20 THEN 'C'
#             ELSE 'D'
#         END AS segment 
#     FROM checks
#     GROUP BY UserID
#     ) AS r
# ON l.UserID = r.UserID
# GROUP BY r.segment
# ORDER BY SUM_segment DESC

In [ ]:
D	250551286
C	223388252
B	72257465
A	65057410

###### ОТВЕТ : 250551286

## 12

In [ ]:
Вернемся к таблице AirBnb. Предположим, что в выборе жилья нас интересует только два параметра: 
    
наличие кухни (kitchen) и гибкой системы отмены (flexible), причем первый в приоритете.

Создайте с помощью оператора CASE колонку с обозначением группы, в которую попадает жилье из таблицы listings:

'good', если в удобствах (amenities) присутствует кухня и система отмены (cancellation_policy) гибкая
'ok', если в удобствах есть кухня, но система отмены не гибкая
'not ok' во всех остальных случаях
Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, 

в качестве ответа укажите host_id первой строки.

Обратите внимание, что cancellation_policy - это отдельная колонка, по ней необходимо смотреть систему отмены

Hint: документация по функции multiSearchAnyCaseInsensitive(колонка, ['искомая_подстрока'])

In [ ]:
SELECT host_id,
    CASE
        WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) = True
        AND multiSearchAnyCaseInsensitive(cancellation_policy, ['flexible']) = True THEN 'good' 
        WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) = True 
        AND multiSearchAnyCaseInsensitive(cancellation_policy, ['flexible']) = 0 THEN 'ok'
        ELSE 'not ok'
    END AS Good_Ok_Not 
FROM
    listings

ORDER BY Good_Ok_Not ASC

LIMIT 5

In [ ]:
27607502	good
75398823	good
10780623	good
6637229	good
199462820	good

###### ОТВЕТ: 27607502